In [1]:
import torch

from botorch.fit import fit_gpytorch_model
from botorch.models import SingleTaskGP
from botorch.test_functions import Rastrigin
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import qExpectedImprovement
from botorch.optim import optimize_acqf

objective_func = Rastrigin(dim=2, negate=True)

In [2]:
torch.manual_seed(1)
train_x = torch.rand(100, 2, dtype=torch.double)
train_obj = objective_func(train_x).unsqueeze(-1)
model = SingleTaskGP(train_X=train_x, train_Y=train_obj)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_model(mll);

best_value = train_obj.max()
qEI = qExpectedImprovement(model=model, best_f=best_value)

new_point_analytic, _ = optimize_acqf(
    acq_function=qEI,
    bounds=torch.tensor([[-5.12] * 2, [5.12] * 2]),
    q=5,
    num_restarts=20,
    raw_samples=512,
    options={"batch_limit": 20, "init_batch_limit": 20},
)
new_point_analytic

tensor([[-0.1368,  1.0684],
        [-0.3418,  1.2239],
        [-5.1200, -1.8558],
        [ 3.2311, -5.0935],
        [-0.1144, -0.1134]])